# Distribution of NO<sub>2</sub> by CAMS and TROPOMI

### Main goal

Show side by side the total columns of NO<sub>2</sub> from CAMS and TROPOMI datasets.

### Selected period and study area
* Period: 21/06/2021 between 00:00:00 and 18:00:00
* Location: Czech Republic

### TODO
* Add apriori data to calculate component column 

## Imports and supporting functions

In [1]:
"""
Notes:

- The library cartopy needs the following:
sudo apt-get install libproj-dev proj-data proj-bin 
sudo apt-get install libgeos-dev 
sudo pip install cython

- eccodes must be installed to be able to read grib files (Linux)
Reference: https://gist.github.com/MHBalsmeier/a01ad4e07ecf467c90fad2ac7719844a
"""

'\nNotes:\n\n- The library cartopy needs the following:\nsudo apt-get install libproj-dev proj-data proj-bin \nsudo apt-get install libgeos-dev \nsudo pip install cython\n\n- eccodes must be installed to be able to read grib files (Linux)\nReference: https://gist.github.com/MHBalsmeier/a01ad4e07ecf467c90fad2ac7719844a\n'

In [2]:
from sentinelsat.sentinel import SentinelAPI
import cdsapi
import cfgrib
import xarray as xr
import pandas as pd
import numpy as np
from itertools import product
import scipy.interpolate
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import math

In [3]:
%run functions.ipynb

In [4]:
# Hide pandas warning
pd.options.mode.chained_assignment = None

## 1. Define study area

In [5]:
# Define limit latitudes and longitudes
lat_min = 48.5
lat_max = 51.1
lon_min = 12
lon_max = 18.9

# Create bbox
bbox = ((lon_min, lat_min), (lon_max, lat_max))

## 2. Define datasets

In [6]:
# Define component
component = 'nitrogen_dioxide'
component_nom = 'NO2'
component_mol_weight = 46.005 #g/mol

# CAMS: Define period
start_date = '2021-06-21'
end_date = '2021-06-21'

# TROPOMI: Define file and product name
file_name = 'd762cc7a-f037-4fb8-bc04-cac8e7f39f5b'
product_name = 'S5P_NRTI_L2__NO2____20210621T124026_20210621T124526_19112_01_010400_20210621T132518.nc'

"""
# Currently available datasets to download
# CAMS: Define period
start_date = '2021-07-21'
end_date = '2021-07-21'

# TROPOMI: Define file and product name
file_name = '42ceb065-e6e8-4783-9e6b-4a01d266f54d'
product_name = 'S5P_NRTI_L2__NO2____20210721T113737_20210721T114237_19537_02_020200_20210721T124127.nc'
"""

"\n# CAMS: Define period\nstart_date = '2021-07-21'\nend_date = '2021-07-21'\n\n# TROPOMI: Define file and product name\nfile_name = '42ceb065-e6e8-4783-9e6b-4a01d266f54d'\nproduct_name = 'S5P_NRTI_L2__NO2____20210721T113737_20210721T114237_19537_02_020200_20210721T124127.nc'\n"

## 3. TROPOMI data

### Collect and read data as xarray

In [7]:
#TROPOMI_download(file_name, component_nom)
TROPOMI_ds = TROPOMI_read(product_name, component_nom)
TROPOMI_ds

<xarray.Dataset>
Dimensions:                                               (corner: 4, ground_pixel: 450, intensity_offset_polynomial_exponents: 1, layer: 34, polynomial_exponents: 6, scanline: 373, time: 1, vertices: 2)
Coordinates:
  * scanline                                              (scanline) float64 ...
  * ground_pixel                                          (ground_pixel) float64 ...
  * time                                                  (time) datetime64[ns] ...
  * corner                                                (corner) float64 0....
  * polynomial_exponents                                  (polynomial_exponents) float64 ...
  * intensity_offset_polynomial_exponents                 (intensity_offset_polynomial_exponents) float64 ...
  * layer                                                 (layer) float64 0.0...
  * vertices                                              (vertices) float64 ...
    latitude                                              (time, scanline, ground_pixel) float32 ...
    longitude                                             (time, scanline, ground_pixel) float32 ...
Data variables:
    delta_time                                            (time, scanline) datetime64[ns] ...
    time_utc                                              (time, scanline) object ...
    qa_value                                              (time, scanline, ground_pixel) float32 ...
    TROPOMI_column                                        (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision         (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision_kernel  (time, scanline, ground_pixel) float32 ...
    averaging_kernel                                      (time, scanline, ground_pixel, layer) float32 ...
    air_mass_factor_troposphere                           (time, scanline, ground_pixel) float32 ...
    air_mass_factor_total                                 (time, scanline, ground_pixel) float32 ...
    tm5_tropopause_layer_index                            (time, scanline, ground_pixel) float64 ...
    tm5_constant_a                                        (layer, vertices) float32 ...
    tm5_constant_b                                        (layer, vertices) float32 ...

### Calculate pressure

In [8]:
# Read support data to calculate pressure
support_ds = xr.open_dataset('data/tropomi/' + component_nom + '/' + product_name, group = 'PRODUCT/SUPPORT_DATA/INPUT_DATA')

# Calculate pressure as p = ap + b * ps
# Units: ap(Pa) + b(none) * ps(Pa) -> To hPa
pressure_upper_bound = (TROPOMI_ds.tm5_constant_a.sel(vertices = 1) + 
                        TROPOMI_ds.tm5_constant_b.sel(vertices = 1) * support_ds.surface_pressure) / 100

pressure_lower_bound = (TROPOMI_ds.tm5_constant_a.sel(vertices = 0) + 
                        TROPOMI_ds.tm5_constant_b.sel(vertices = 0) * support_ds.surface_pressure) / 100

pressure_half = (pressure_upper_bound + pressure_lower_bound) / 2

TROPOMI_ds = TROPOMI_ds.assign(pressure_up = pressure_upper_bound)
TROPOMI_ds = TROPOMI_ds.assign(pressure_low = pressure_lower_bound)
TROPOMI_ds = TROPOMI_ds.assign(pressure = pressure_half)

TROPOMI_ds

<xarray.Dataset>
Dimensions:                                               (corner: 4, ground_pixel: 450, intensity_offset_polynomial_exponents: 1, layer: 34, polynomial_exponents: 6, scanline: 373, time: 1, vertices: 2)
Coordinates:
  * ground_pixel                                          (ground_pixel) float64 ...
  * scanline                                              (scanline) float64 ...
  * time                                                  (time) datetime64[ns] ...
  * corner                                                (corner) float64 0....
  * polynomial_exponents                                  (polynomial_exponents) float64 ...
  * intensity_offset_polynomial_exponents                 (intensity_offset_polynomial_exponents) float64 ...
  * layer                                                 (layer) float64 0.0...
  * vertices                                              (vertices) float64 ...
    latitude                                              (time, scanline, ground_pixel) float32 ...
    longitude                                             (time, scanline, ground_pixel) float32 ...
Data variables: (12/15)
    delta_time                                            (time, scanline) datetime64[ns] ...
    time_utc                                              (time, scanline) object ...
    qa_value                                              (time, scanline, ground_pixel) float32 ...
    TROPOMI_column                                        (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision         (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision_kernel  (time, scanline, ground_pixel) float32 ...
    ...                                                    ...
    tm5_tropopause_layer_index                            (time, scanline, ground_pixel) float64 ...
    tm5_constant_a                                        (layer, vertices) float32 ...
    tm5_constant_b                                        (layer, vertices) float32 ...
    pressure_up                                           (layer, time, scanline, ground_pixel) float32 ...
    pressure_low                                          (layer, time, scanline, ground_pixel) float32 ...
    pressure                                              (layer, time, scanline, ground_pixel) float32 ...

### Reduce data to study area

In [9]:
"""
# Create dataframe with scanlines and ground pixels
TROPOMI_coords_df = []
TROPOMI_coords_df = pd.DataFrame(list(product(TROPOMI_ds.ground_pixel.values, TROPOMI_ds.scanline.values)), 
                          columns = ['ground_pixel', 'scanline'])

# Find corresponding latitudes and longitudes                          
for index, row in TROPOMI_coords_df.iterrows():
    
    TROPOMI_coords_df.loc[index,'latitude'] = TROPOMI_ds.latitude.sel(
                                              scanline = TROPOMI_coords_df['scanline'].loc[index], 
                                              ground_pixel = TROPOMI_coords_df['ground_pixel'].loc[index],
                                              method = None).values
                                        
    TROPOMI_coords_df.loc[index,'longitude'] = TROPOMI_ds.longitude.sel(
                                               scanline = TROPOMI_coords_df['scanline'].loc[index], 
                                               ground_pixel = TROPOMI_coords_df['ground_pixel'].loc[index],
                                               method = None).values

# Save as csv
TROPOMI_coords_df.to_csv('data/tropomi/TROPOMI_coords_df_' + product_name + '.csv', index = False)
"""

"\n# Create dataframe with scanlines and ground pixels\nTROPOMI_coords_df = []\nTROPOMI_coords_df = pd.DataFrame(list(product(TROPOMI_ds.ground_pixel.values, TROPOMI_ds.scanline.values)), \n                          columns = ['ground_pixel', 'scanline'])\n\n# Find corresponding latitudes and longitudes                          \nfor index, row in TROPOMI_coords_df.iterrows():\n    \n    TROPOMI_coords_df.loc[index,'latitude'] = TROPOMI_ds.latitude.sel(\n                                              scanline = TROPOMI_coords_df['scanline'].loc[index], \n                                              ground_pixel = TROPOMI_coords_df['ground_pixel'].loc[index],\n                                              method = None).values\n                                        \n    TROPOMI_coords_df.loc[index,'longitude'] = TROPOMI_ds.longitude.sel(\n                                               scanline = TROPOMI_coords_df['scanline'].loc[index], \n                                             

In [10]:
# Read csv
TROPOMI_coords_df = pd.read_csv('data/tropomi/TROPOMI_coords_df_' + product_name + '.csv')

# Set limits
TROPOMI_coords_df = TROPOMI_coords_df[(TROPOMI_coords_df['latitude'] >= lat_min) & (TROPOMI_coords_df['latitude'] <= lat_max)]
TROPOMI_coords_df = TROPOMI_coords_df[(TROPOMI_coords_df['longitude'] >= lon_min) & (TROPOMI_coords_df['longitude'] <= lon_max)]

TROPOMI_coords_df

,ground_pixel,scanline,latitude,longitude
144534,387.0,183.0,48.546898,12.072847
144535,387.0,184.0,48.595726,12.062323
144536,387.0,185.0,48.644573,12.051785
144537,387.0,186.0,48.693401,12.041157
144538,387.0,187.0,48.742229,12.030693
...,...,...,...,...
164355,440.0,235.0,50.872116,18.894257
164356,440.0,236.0,50.920200,18.891136
164357,440.0,237.0,50.968403,18.888002
164358,440.0,238.0,51.016579,18.885010


In [11]:
plt.scatter(TROPOMI_coords_df['ground_pixel'], TROPOMI_coords_df['scanline'], s = 3)

In [12]:
# Get scanline and ground pixel coordinates
scanline_coords = np.unique(TROPOMI_coords_df['scanline'].values).tolist()
ground_pixel_coords = np.unique(TROPOMI_coords_df['ground_pixel'].values).tolist()

In [13]:
# Set limits
TROPOMI_ds = TROPOMI_ds.sel(scanline = scanline_coords, ground_pixel = ground_pixel_coords)
TROPOMI_ds

<xarray.Dataset>
Dimensions:                                               (corner: 4, ground_pixel: 54, intensity_offset_polynomial_exponents: 1, layer: 34, polynomial_exponents: 6, scanline: 58, time: 1, vertices: 2)
Coordinates:
  * ground_pixel                                          (ground_pixel) float64 ...
  * scanline                                              (scanline) float64 ...
  * time                                                  (time) datetime64[ns] ...
  * corner                                                (corner) float64 0....
  * polynomial_exponents                                  (polynomial_exponents) float64 ...
  * intensity_offset_polynomial_exponents                 (intensity_offset_polynomial_exponents) float64 ...
  * layer                                                 (layer) float64 0.0...
  * vertices                                              (vertices) float64 ...
    latitude                                              (time, scanline, ground_pixel) float32 ...
    longitude                                             (time, scanline, ground_pixel) float32 ...
Data variables: (12/15)
    delta_time                                            (time, scanline) datetime64[ns] ...
    time_utc                                              (time, scanline) object ...
    qa_value                                              (time, scanline, ground_pixel) float32 ...
    TROPOMI_column                                        (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision         (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision_kernel  (time, scanline, ground_pixel) float32 ...
    ...                                                    ...
    tm5_tropopause_layer_index                            (time, scanline, ground_pixel) float64 ...
    tm5_constant_a                                        (layer, vertices) float32 ...
    tm5_constant_b                                        (layer, vertices) float32 ...
    pressure_up                                           (layer, time, scanline, ground_pixel) float32 ...
    pressure_low                                          (layer, time, scanline, ground_pixel) float32 ...
    pressure                                              (layer, time, scanline, ground_pixel) float32 ...

In [14]:
# Create dataframe with scanlines and ground pixels
TROPOMI_coords_df_2 = []
TROPOMI_coords_df_2 = pd.DataFrame(list(product(TROPOMI_ds.ground_pixel.values, TROPOMI_ds.scanline.values)), 
                          columns = ['ground_pixel', 'scanline'])

# Find corresponding latitudes and longitudes                          
for index, row in TROPOMI_coords_df_2.iterrows():
    TROPOMI_coords_df_2.loc[index,'latitude'] = TROPOMI_ds.latitude.sel(
                                              scanline = TROPOMI_coords_df_2['scanline'].loc[index], 
                                              ground_pixel = TROPOMI_coords_df_2['ground_pixel'].loc[index],
                                              method = None).values
                                        
    TROPOMI_coords_df_2.loc[index,'longitude'] = TROPOMI_ds.longitude.sel(
                                               scanline = TROPOMI_coords_df_2['scanline'].loc[index], 
                                               ground_pixel = TROPOMI_coords_df_2['ground_pixel'].loc[index],
                                               method = None).values

# Save as csv
TROPOMI_coords_df_2

,ground_pixel,scanline,latitude,longitude
0,387.0,182.0,48.498062,12.083351
1,387.0,183.0,48.546898,12.072847
2,387.0,184.0,48.595726,12.062323
3,387.0,185.0,48.644573,12.051785
4,387.0,186.0,48.693401,12.041157
...,...,...,...,...
3127,440.0,235.0,50.872116,18.894257
3128,440.0,236.0,50.920200,18.891136
3129,440.0,237.0,50.968403,18.888002
3130,440.0,238.0,51.016579,18.885010


In [15]:
# ERROR: The selected values for the study area are not sliced properly! (there are extra points)
plt.scatter(TROPOMI_coords_df_2['ground_pixel'], TROPOMI_coords_df_2['scanline'], s = 3)

### Calculate column averaging kernels

In [16]:
kernel_column_da = xr.where(TROPOMI_ds.layer > TROPOMI_ds.tm5_tropopause_layer_index, 0, 
                            TROPOMI_ds.averaging_kernel * (TROPOMI_ds.air_mass_factor_total / 
                            TROPOMI_ds.air_mass_factor_troposphere))

TROPOMI_ds = TROPOMI_ds.assign(kernel_column = kernel_column_da)

TROPOMI_ds

<xarray.Dataset>
Dimensions:                                               (corner: 4, ground_pixel: 54, intensity_offset_polynomial_exponents: 1, layer: 34, polynomial_exponents: 6, scanline: 58, time: 1, vertices: 2)
Coordinates:
  * ground_pixel                                          (ground_pixel) float64 ...
  * scanline                                              (scanline) float64 ...
  * time                                                  (time) datetime64[ns] ...
  * corner                                                (corner) float64 0....
  * polynomial_exponents                                  (polynomial_exponents) float64 ...
  * intensity_offset_polynomial_exponents                 (intensity_offset_polynomial_exponents) float64 ...
  * layer                                                 (layer) float64 0.0...
  * vertices                                              (vertices) float64 ...
    latitude                                              (time, scanline, ground_pixel) float32 ...
    longitude                                             (time, scanline, ground_pixel) float32 ...
Data variables: (12/16)
    delta_time                                            (time, scanline) datetime64[ns] ...
    time_utc                                              (time, scanline) object ...
    qa_value                                              (time, scanline, ground_pixel) float32 ...
    TROPOMI_column                                        (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision         (time, scanline, ground_pixel) float32 ...
    nitrogendioxide_tropospheric_column_precision_kernel  (time, scanline, ground_pixel) float32 ...
    ...                                                    ...
    tm5_constant_a                                        (layer, vertices) float32 ...
    tm5_constant_b                                        (layer, vertices) float32 ...
    pressure_up                                           (layer, time, scanline, ground_pixel) float32 ...
    pressure_low                                          (layer, time, scanline, ground_pixel) float32 ...
    pressure                                              (layer, time, scanline, ground_pixel) float32 ...
    kernel_column                                         (layer, time, scanline, ground_pixel) float32 ...

### Convert TROPOMI data units (From mol/m<sup>2</sup> to molecules/cm<sup>2</sup>)

In [17]:
TROPOMI_ds['TROPOMI_column'] = TROPOMI_ds.TROPOMI_column * 6.02214*10**19

## 4. CAMS data

### Collect and read data as xarray

In [18]:
#CAMS_download(start_date, end_date, component, component_nom)
CAMS_ds = CAMS_read(start_date, end_date, component, component_nom)
CAMS_ds

<xarray.Dataset>
Dimensions:     (hybrid: 137, latitude: 451, longitude: 900, step: 4)
Coordinates:
    time        datetime64[ns] ...
  * step        (step) timedelta64[ns] 00:00:00 06:00:00 12:00:00 18:00:00
  * hybrid      (hybrid) float64 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
  * latitude    (latitude) float64 -90.0 -89.6 -89.2 -88.8 ... 89.2 89.6 90.0
  * longitude   (longitude) float64 -180.0 -179.6 -179.2 ... 178.8 179.2 179.6
    valid_time  (step) datetime64[ns] ...
Data variables:
    component   (step, hybrid, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-07-26T18:44 GRIB to CDM+CF via cfgrib-0.9.9...

### Collect 137 levels model data

In [19]:
# Read csv table with 137 levels
CAMS_levels = pd.read_csv('data/ads/137-levels.csv')

# Drop first row and set n as index hybrid
CAMS_levels = CAMS_levels.drop(0).reset_index(drop = True)
CAMS_levels = CAMS_levels.set_index('n')
CAMS_levels.index.names = ['hybrid']

# Change important columns to numeric
CAMS_levels['pf [hPa]'] = pd.to_numeric(CAMS_levels['pf [hPa]'])
CAMS_levels['Geopotential Altitude [m]'] = pd.to_numeric(CAMS_levels['Geopotential Altitude [m]'])
CAMS_levels['Density [kg/m^3]'] = pd.to_numeric(CAMS_levels['Density [kg/m^3]'])

# Calculate half pressures
CAMS_levels['ph-diff [Pa]'] = CAMS_levels['ph [hPa]'].diff(1) * 100

# Add half pressure difference for the top of the atmosphere (first row) manually
CAMS_levels['ph-diff [Pa]'].iloc[0] = CAMS_levels['ph [hPa]'].iloc[0] * 100

# Calculate difference from geopotential altitude
CAMS_levels['Depth [m]'] = CAMS_levels['Geopotential Altitude [m]'].diff(-1)

# Add layer depth to last row manually
CAMS_levels['Depth [m]'].iloc[-1] = CAMS_levels['Geopotential Altitude [m]'].iloc[-1]

CAMS_levels

,a [Pa],b,ph [hPa],pf [hPa],Geopotential Altitude [m],Geometric Altitude [m],Temperature [K],Density [kg/m^3],ph-diff [Pa],Depth [m]
hybrid,,,,,,,,,,
1,2.000365,0.000000,0.0200,0.0100,79301.79,80301.65,198.05,0.000018,2.00,5580.21
2,3.102241,0.000000,0.0310,0.0255,73721.58,74584.91,209.21,0.000042,1.10,2605.83
3,4.666084,0.000000,0.0467,0.0388,71115.75,71918.79,214.42,0.000063,1.57,2497.32
4,6.827977,0.000000,0.0683,0.0575,68618.43,69365.77,221.32,0.000090,2.16,2407.44
5,9.746966,0.000000,0.0975,0.0829,66210.99,66906.53,228.06,0.000127,2.92,2320.96
...,...,...,...,...,...,...,...,...,...,...
133,62.781250,0.988500,1002.2250,1000.5165,106.54,106.54,287.46,1.212498,341.69,27.50
134,22.835938,0.991984,1005.3562,1003.7906,79.04,79.04,287.64,1.215710,313.12,25.12
135,3.757813,0.995003,1008.2239,1006.7900,53.92,53.92,287.80,1.218650,286.77,22.96


### Collect pressure surface data
    Not available

### Reduce data to study area

In [20]:
# Get nearest longitude and latitude to bbox
lon_min_index = nearest_neighbour(CAMS_ds.longitude.data, bbox[0][0])
lon_max_index = nearest_neighbour(CAMS_ds.longitude.data, bbox[1][0])
lat_min_index = nearest_neighbour(CAMS_ds.latitude.data, bbox[0][1])
lat_max_index = nearest_neighbour(CAMS_ds.latitude.data, bbox[1][1])

# Define slices
slice_lat = slice(lat_min_index, lat_max_index + 1)
slice_lon = slice(lon_min_index, lon_max_index + 1)

# Set limits
CAMS_ds = CAMS_ds.isel(longitude = slice_lon, latitude = slice_lat)

CAMS_ds

<xarray.Dataset>
Dimensions:     (hybrid: 137, latitude: 8, longitude: 18, step: 4)
Coordinates:
    time        datetime64[ns] 2021-06-21
  * step        (step) timedelta64[ns] 00:00:00 06:00:00 12:00:00 18:00:00
  * hybrid      (hybrid) float64 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
  * latitude    (latitude) float64 48.4 48.8 49.2 49.6 50.0 50.4 50.8 51.2
  * longitude   (longitude) float64 12.0 12.4 12.8 13.2 ... 17.6 18.0 18.4 18.8
    valid_time  (step) datetime64[ns] 2021-06-21 ... 2021-06-21T18:00:00
Data variables:
    component   (step, hybrid, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-07-26T18:44 GRIB to CDM+CF via cfgrib-0.9.9...

### Linear interpolation of CAMS data to 100x100

In [21]:
#CAMS_ds =  CAMS_interpolation(CAMS_ds, TROPOMI_ds, lon_min, lon_max, lat_min, lat_max, component_nom)

### Calculate NO<sub>2</sub> partial column (kg/m<sup>2</sup>) above each CAMS half level

In [22]:
g = 9.81
gi = 1/g #m/s2

da = []
da_hybrid = []

# Initialize partial columns at the top of the atmosphere as 0
PC_0 = 0 * CAMS_ds.sel(hybrid = 1)
da_hybrid.append(PC_0)
CAMS_ds_PC = xr.concat(da_hybrid, dim = 'hybrid')

for hybrid in range(1, CAMS_ds.hybrid.size):

    PC_last = CAMS_ds_PC.sel(hybrid = hybrid)
    component = CAMS_ds.sel(hybrid = hybrid + 1)
    
    # Units: Component(kg/kg) * ph-diff(Pa = kg/m*s2)) * m/s2 -> To kg/m2
    PC_da = PC_last + component * CAMS_levels['ph-diff [Pa]'].loc[hybrid] * gi
    
    da_hybrid.append(PC_da)
    CAMS_ds_PC = xr.concat(da_hybrid, pd.Index(range(1, hybrid + 2), name = 'hybrid'))

CAMS_ds = CAMS_ds_PC
CAMS_ds

<xarray.Dataset>
Dimensions:     (hybrid: 137, latitude: 8, longitude: 18, step: 4)
Coordinates:
    time        datetime64[ns] 2021-06-21
  * step        (step) timedelta64[ns] 00:00:00 06:00:00 12:00:00 18:00:00
  * hybrid      (hybrid) int64 1 2 3 4 5 6 7 8 ... 131 132 133 134 135 136 137
  * latitude    (latitude) float64 48.4 48.8 49.2 49.6 50.0 50.4 50.8 51.2
  * longitude   (longitude) float64 12.0 12.4 12.8 13.2 ... 17.6 18.0 18.4 18.8
    valid_time  (step) datetime64[ns] 2021-06-21 ... 2021-06-21T18:00:00
Data variables:
    component   (hybrid, step, latitude, longitude) float32 0.0 ... 8.272e-06
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-07-26T18:44 GRIB to CDM+CF via cfgrib-0.9.9...

### Convert CAMS data units (kg/m<sup>2</sup> to molecules/cm<sup>2</sup>)

In [23]:
# From kg/m2 to molecules/cm2
NA = 6.022*10**23
CAMS_ds = (CAMS_ds * NA * 1000) / (10000 * component_mol_weight)

CAMS_ds

<xarray.Dataset>
Dimensions:     (hybrid: 137, latitude: 8, longitude: 18, step: 4)
Coordinates:
    time        datetime64[ns] 2021-06-21
  * step        (step) timedelta64[ns] 00:00:00 06:00:00 12:00:00 18:00:00
  * hybrid      (hybrid) int64 1 2 3 4 5 6 7 8 ... 131 132 133 134 135 136 137
  * latitude    (latitude) float64 48.4 48.8 49.2 49.6 50.0 50.4 50.8 51.2
  * longitude   (longitude) float64 12.0 12.4 12.8 13.2 ... 17.6 18.0 18.4 18.8
    valid_time  (step) datetime64[ns] 2021-06-21 ... 2021-06-21T18:00:00
Data variables:
    component   (hybrid, step, latitude, longitude) float32 0.0 ... 1.083e+16
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-07-26T18:44 GRIB to CDM+CF via cfgrib-0.9.9...

## 5. Apply averaging kernels

### Transform TROPOMI dataset into a dataframe

In [24]:
TROPOMI_df = TROPOMI_ds.to_dataframe()

TROPOMI_df = TROPOMI_df.drop(columns=['nitrogendioxide_tropospheric_column_precision', 
                                      'nitrogendioxide_tropospheric_column_precision_kernel',
                                      'averaging_kernel',
                                      'air_mass_factor_troposphere',
                                      'air_mass_factor_total',
                                      'tm5_tropopause_layer_index',
                                      'tm5_constant_a',
                                      'tm5_constant_b',
                                      'pressure_up',
                                      'pressure_low',
                                      'time_utc'])

# Pass NaNs to data with qa_value under 0.5
TROPOMI_df.loc[TROPOMI_df['qa_value'] < 0.5, ['TROPOMI_column', 'kernel_column']] = float('NaN')

# Drop levels and reorder
TROPOMI_df = TROPOMI_df.droplevel(['corner', 'intensity_offset_polynomial_exponents', 'polynomial_exponents']).drop_duplicates()
TROPOMI_df = TROPOMI_df.groupby(by = ['layer', 'pressure', 'scanline', 'ground_pixel', 'time', 'delta_time']).mean()
TROPOMI_df = TROPOMI_df.reset_index(level = ['layer', 'delta_time'])
TROPOMI_df

layer              delta_time  \
pressure   scanline ground_pixel time                                        
872.439514 229.0    418.0        2021-06-21    0.0 2021-06-21 12:43:32.952   
879.006348 192.0    401.0        2021-06-21    0.0 2021-06-21 12:43:01.872   
880.255981 192.0    400.0        2021-06-21    0.0 2021-06-21 12:43:01.872   
883.040039 193.0    400.0        2021-06-21    0.0 2021-06-21 12:43:02.712   
883.846741 228.0    418.0        2021-06-21    0.0 2021-06-21 12:43:32.112   
...                                            ...                     ...   
0.093045   239.0    436.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    437.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    438.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    439.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    440.0        2021-06-21   33.0 2021-06-21 12:43:41.351   

                                              latitude  longitude  qa_value  \
pressure   scanline ground_pixel time                                         
872.439514 229.0    418.0        2021-06-21  50.744083  15.669636      0.74   
879.006348 192.0    401.0        2021-06-21  48.992073  13.550930      1.00   
880.255981 192.0    400.0        2021-06-21  48.992691  13.429176      1.00   
883.040039 193.0    400.0        2021-06-21  49.041451  13.419986      1.00   
883.846741 228.0    418.0        2021-06-21  50.695518  15.675993      0.74   
...                                                ...        ...       ...   
0.093045   239.0    436.0        2021-06-21  51.093616  18.428249      1.00   
                    437.0        2021-06-21  51.086693  18.539761      1.00   
                    438.0        2021-06-21  51.079575  18.652542      1.00   
                    439.0        2021-06-21  51.072258  18.766626      1.00   
                    440.0        2021-06-21  51.064732  18.882040      1.00   

                                             TROPOMI_column  kernel_column  
pressure   scanline ground_pixel time                                       
872.439514 229.0    418.0        2021-06-21    1.327695e+15       0.048624  
879.006348 192.0    401.0        2021-06-21    9.852745e+14       0.109274  
880.255981 192.0    400.0        2021-06-21    1.352566e+15       0.156241  
883.040039 193.0    400.0        2021-06-21    1.335530e+15       0.158627  
883.846741 228.0    418.0        2021-06-21    1.244296e+15       0.070985  
...                                                     ...            ...  
0.093045   239.0    436.0        2021-06-21    1.716015e+15       0.000000  
                    437.0        2021-06-21    1.962079e+15       0.000000  
                    438.0        2021-06-21    1.898344e+15       0.000000  
                    439.0        2021-06-21    2.213572e+15       0.000000  
                    440.0        2021-06-21    2.239043e+15       0.000000  

[106488 rows x 7 columns]

### Combine data (nearest neighbours)

In [25]:
"""
match_df = TROPOMI_df

CAMS_latitudes = CAMS_ds.latitude.data
CAMS_longitudes = CAMS_ds.longitude.data
CAMS_pressures = CAMS_levels['pf [hPa]'].to_numpy()
CAMS_times = CAMS_ds.valid_time.data

match_df['lat_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_latitudes, row['latitude']), axis = 1)
match_df['lon_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_longitudes, row['longitude']), axis = 1)
match_df['lay_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_pressures, row['pressure']), axis = 1)
match_df['step_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_times, row['delta_time']), axis = 1)
match_df['CAMS_time'] = match_df.apply(lambda row: CAMS_ds.valid_time[row['step_index']].values, axis = 1)

match_df['CAMS_component'] = match_df.apply(lambda row: CAMS_ds.component.isel(
                                                        hybrid = int(row['lay_index']) - 1, 
                                                        latitude = int(row['lat_index']), 
                                                        longitude = int(row['lon_index']), 
                                                        step = int(row['step_index'])).values, 
                                                        axis = 1)

match_df
"""

"\nmatch_df = TROPOMI_df\n\nCAMS_latitudes = CAMS_ds.latitude.data\nCAMS_longitudes = CAMS_ds.longitude.data\nCAMS_pressures = CAMS_levels['pf [hPa]'].to_numpy()\nCAMS_times = CAMS_ds.valid_time.data\n\nmatch_df['lat_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_latitudes, row['latitude']), axis = 1)\nmatch_df['lon_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_longitudes, row['longitude']), axis = 1)\nmatch_df['lay_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_pressures, row['pressure']), axis = 1)\nmatch_df['step_index'] = match_df.apply(lambda row: nearest_neighbour(CAMS_times, row['delta_time']), axis = 1)\nmatch_df['CAMS_time'] = match_df.apply(lambda row: CAMS_ds.valid_time[row['step_index']].values, axis = 1)\n\nmatch_df['CAMS_component'] = match_df.apply(lambda row: CAMS_ds.component.isel(\n                                                        hybrid = int(row['lay_index']) - 1, \n                                                       

### Combine data (nearest neighbours + vertical interpolation)

#### Prepare dataframe for interpolation

In [26]:
match_df = TROPOMI_df
match_df

layer              delta_time  \
pressure   scanline ground_pixel time                                        
872.439514 229.0    418.0        2021-06-21    0.0 2021-06-21 12:43:32.952   
879.006348 192.0    401.0        2021-06-21    0.0 2021-06-21 12:43:01.872   
880.255981 192.0    400.0        2021-06-21    0.0 2021-06-21 12:43:01.872   
883.040039 193.0    400.0        2021-06-21    0.0 2021-06-21 12:43:02.712   
883.846741 228.0    418.0        2021-06-21    0.0 2021-06-21 12:43:32.112   
...                                            ...                     ...   
0.093045   239.0    436.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    437.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    438.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    439.0        2021-06-21   33.0 2021-06-21 12:43:41.351   
                    440.0        2021-06-21   33.0 2021-06-21 12:43:41.351   

                                              latitude  longitude  qa_value  \
pressure   scanline ground_pixel time                                         
872.439514 229.0    418.0        2021-06-21  50.744083  15.669636      0.74   
879.006348 192.0    401.0        2021-06-21  48.992073  13.550930      1.00   
880.255981 192.0    400.0        2021-06-21  48.992691  13.429176      1.00   
883.040039 193.0    400.0        2021-06-21  49.041451  13.419986      1.00   
883.846741 228.0    418.0        2021-06-21  50.695518  15.675993      0.74   
...                                                ...        ...       ...   
0.093045   239.0    436.0        2021-06-21  51.093616  18.428249      1.00   
                    437.0        2021-06-21  51.086693  18.539761      1.00   
                    438.0        2021-06-21  51.079575  18.652542      1.00   
                    439.0        2021-06-21  51.072258  18.766626      1.00   
                    440.0        2021-06-21  51.064732  18.882040      1.00   

                                             TROPOMI_column  kernel_column  
pressure   scanline ground_pixel time                                       
872.439514 229.0    418.0        2021-06-21    1.327695e+15       0.048624  
879.006348 192.0    401.0        2021-06-21    9.852745e+14       0.109274  
880.255981 192.0    400.0        2021-06-21    1.352566e+15       0.156241  
883.040039 193.0    400.0        2021-06-21    1.335530e+15       0.158627  
883.846741 228.0    418.0        2021-06-21    1.244296e+15       0.070985  
...                                                     ...            ...  
0.093045   239.0    436.0        2021-06-21    1.716015e+15       0.000000  
                    437.0        2021-06-21    1.962079e+15       0.000000  
                    438.0        2021-06-21    1.898344e+15       0.000000  
                    439.0        2021-06-21    2.213572e+15       0.000000  
                    440.0        2021-06-21    2.239043e+15       0.000000  

[106488 rows x 7 columns]

In [27]:
# Create index that includes CAMS pressure levels for all the locations in TROPOMI
new_index = pd.MultiIndex.from_product([CAMS_levels['pf [hPa]'].to_numpy(), 
                                        match_df.index.levels[1], 
                                        match_df.index.levels[2],
                                        match_df.index.levels[3]],
                                        names = ['pressure', 'scanline', 'ground_pixel', 'time'])

new_index

MultiIndex([(     0.01, 182.0, 387.0, '2021-06-21'),
            (     0.01, 182.0, 388.0, '2021-06-21'),
            (     0.01, 182.0, 389.0, '2021-06-21'),
            (     0.01, 182.0, 390.0, '2021-06-21'),
            (     0.01, 182.0, 391.0, '2021-06-21'),
            (     0.01, 182.0, 392.0, '2021-06-21'),
            (     0.01, 182.0, 393.0, '2021-06-21'),
            (     0.01, 182.0, 394.0, '2021-06-21'),
            (     0.01, 182.0, 395.0, '2021-06-21'),
            (     0.01, 182.0, 396.0, '2021-06-21'),
            ...
            (1012.0494, 239.0, 431.0, '2021-06-21'),
            (1012.0494, 239.0, 432.0, '2021-06-21'),
            (1012.0494, 239.0, 433.0, '2021-06-21'),
            (1012.0494, 239.0, 434.0, '2021-06-21'),
            (1012.0494, 239.0, 435.0, '2021-06-21'),
            (1012.0494, 239.0, 436.0, '2021-06-21'),
            (1012.0494, 239.0, 437.0, '2021-06-21'),
            (1012.0494, 239.0, 438.0, '2021-06-21'),
            (1012.0494, 239.0,

In [28]:
# Append original and new indexes and reindex dataframe
match_df = match_df.reindex(match_df.index.append(new_index))
match_df

layer              delta_time  \
pressure    scanline ground_pixel time                                        
872.439514  229.0    418.0        2021-06-21    0.0 2021-06-21 12:43:32.952   
879.006348  192.0    401.0        2021-06-21    0.0 2021-06-21 12:43:01.872   
880.255981  192.0    400.0        2021-06-21    0.0 2021-06-21 12:43:01.872   
883.040039  193.0    400.0        2021-06-21    0.0 2021-06-21 12:43:02.712   
883.846741  228.0    418.0        2021-06-21    0.0 2021-06-21 12:43:32.112   
...                                             ...                     ...   
1012.049400 239.0    436.0        2021-06-21    NaN                     NaT   
                     437.0        2021-06-21    NaN                     NaT   
                     438.0        2021-06-21    NaN                     NaT   
                     439.0        2021-06-21    NaN                     NaT   
                     440.0        2021-06-21    NaN                     NaT   

                                               latitude  longitude  qa_value  \
pressure    scanline ground_pixel time                                         
872.439514  229.0    418.0        2021-06-21  50.744083  15.669636      0.74   
879.006348  192.0    401.0        2021-06-21  48.992073  13.550930      1.00   
880.255981  192.0    400.0        2021-06-21  48.992691  13.429176      1.00   
883.040039  193.0    400.0        2021-06-21  49.041451  13.419986      1.00   
883.846741  228.0    418.0        2021-06-21  50.695518  15.675993      0.74   
...                                                 ...        ...       ...   
1012.049400 239.0    436.0        2021-06-21        NaN        NaN       NaN   
                     437.0        2021-06-21        NaN        NaN       NaN   
                     438.0        2021-06-21        NaN        NaN       NaN   
                     439.0        2021-06-21        NaN        NaN       NaN   
                     440.0        2021-06-21        NaN        NaN       NaN   

                                              TROPOMI_column  kernel_column  
pressure    scanline ground_pixel time                                       
872.439514  229.0    418.0        2021-06-21    1.327695e+15       0.048624  
879.006348  192.0    401.0        2021-06-21    9.852745e+14       0.109274  
880.255981  192.0    400.0        2021-06-21    1.352566e+15       0.156241  
883.040039  193.0    400.0        2021-06-21    1.335530e+15       0.158627  
883.846741  228.0    418.0        2021-06-21    1.244296e+15       0.070985  
...                                                      ...            ...  
1012.049400 239.0    436.0        2021-06-21             NaN            NaN  
                     437.0        2021-06-21             NaN            NaN  
                     438.0        2021-06-21             NaN            NaN  
                     439.0        2021-06-21             NaN            NaN  
                     440.0        2021-06-21             NaN            NaN  

[535572 rows x 7 columns]

In [29]:
# Sort index
match_df = match_df.sort_index()

# Reset index
match_df = match_df.reset_index()
match_df

,pressure,scanline,ground_pixel,time,layer,delta_time,latitude,longitude,qa_value,TROPOMI_column,kernel_column
0,0.0100,182.0,387.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,0.0100,182.0,388.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,0.0100,182.0,389.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,0.0100,182.0,390.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4,0.0100,182.0,391.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
535567,1012.0494,239.0,436.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
535568,1012.0494,239.0,437.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
535569,1012.0494,239.0,438.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN
535570,1012.0494,239.0,439.0,2021-06-21,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [30]:
# Find latitudes in CAMS rows with scanlines and ground pixels
match_df['latitude'] = match_df.apply(lambda row: float(TROPOMI_coords_df_2[
                                                        (TROPOMI_coords_df_2['scanline'] == row['scanline']) & 
                                                        (TROPOMI_coords_df_2['ground_pixel'] == row['ground_pixel'])]['latitude'])
                                                        if pd.isnull(row['latitude']) else row['latitude'], 
                                                        axis = 1)
                                                        
# Find longitudes in CAMS rows with scanlines and ground pixels
match_df['longitude'] = match_df.apply(lambda row: float(TROPOMI_coords_df_2[
                                                        (TROPOMI_coords_df_2['scanline'] == row['scanline']) & 
                                                        (TROPOMI_coords_df_2['ground_pixel'] == row['ground_pixel'])]['longitude'])
                                                        if pd.isnull(row['longitude']) else row['longitude'], 
                                                        axis = 1)
                                                        
# Find hybrids in CAMS rows from 137 models table
match_df['hybrid'] = match_df.apply(lambda row: nearest_neighbour(CAMS_levels['pf [hPa]'].to_numpy(), row['pressure']) + 1
                                                if pd.isnull(row['TROPOMI_column']) else math.nan, 
                                                axis = 1)

In [31]:
match_df

,pressure,scanline,ground_pixel,time,layer,delta_time,latitude,longitude,qa_value,TROPOMI_column,kernel_column,hybrid
0,0.0100,182.0,387.0,2021-06-21,NaN,NaT,48.498062,12.083351,NaN,NaN,NaN,1.0
1,0.0100,182.0,388.0,2021-06-21,NaN,NaT,48.499416,12.186172,NaN,NaN,NaN,1.0
2,0.0100,182.0,389.0,2021-06-21,NaN,NaT,48.500648,12.290169,NaN,NaN,NaN,1.0
3,0.0100,182.0,390.0,2021-06-21,NaN,NaT,48.501755,12.395366,NaN,NaN,NaN,1.0
4,0.0100,182.0,391.0,2021-06-21,NaN,NaT,48.502731,12.501794,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
535567,1012.0494,239.0,436.0,2021-06-21,NaN,NaT,51.093616,18.428249,NaN,NaN,NaN,137.0
535568,1012.0494,239.0,437.0,2021-06-21,NaN,NaT,51.086693,18.539761,NaN,NaN,NaN,137.0
535569,1012.0494,239.0,438.0,2021-06-21,NaN,NaT,51.079575,18.652542,NaN,NaN,NaN,137.0
535570,1012.0494,239.0,439.0,2021-06-21,NaN,NaT,51.072258,18.766626,NaN,NaN,NaN,137.0


In [32]:
# Get unique timestep
first_delta_time = TROPOMI_ds['delta_time'].isel(scanline = 0, time = 0).values
unique_step = nearest_neighbour(CAMS_ds.valid_time.data, first_delta_time)
unique_time = CAMS_ds.component.isel(step = unique_step).step.values.astype('timedelta64[h]')

# Get CAMS component data at nearby TROPOMI locations (nearest neighbours)
# Do it only for CAMS rows
match_df['CAMS_component'] = match_df.apply(lambda row: CAMS_ds.component.sel(
                                                        hybrid = row['hybrid'], 
                                                        latitude = row['latitude'], 
                                                        longitude = row['longitude'], 
                                                        step = unique_time, method = 'nearest').values 
                                                        if pd.isnull(row['TROPOMI_column']) else math.nan,
                                                        axis = 1)

# Transform 1D-array data to float
match_df['CAMS_component'] = match_df['CAMS_component'].apply(lambda x: float(x))

In [33]:
# Set multiindex again and sort
match_df = match_df.set_index(['time', 'pressure', 'scanline', 'ground_pixel'])
match_df = match_df.sort_values(['time', 'ground_pixel','scanline', 'pressure'], 
                                ascending = [True, True, True, False])

match_df

layer delta_time   latitude  \
time       pressure  scanline ground_pixel                                
2021-06-21 1012.0494 182.0    387.0           NaN        NaT  48.498062   
           1009.5363 182.0    387.0           NaN        NaT  48.498062   
           1006.7900 182.0    387.0           NaN        NaT  48.498062   
           1003.7906 182.0    387.0           NaN        NaT  48.498062   
           1000.5165 182.0    387.0           NaN        NaT  48.498062   
...                                           ...        ...        ...   
           0.0829    239.0    440.0           NaN        NaT  51.064732   
           0.0575    239.0    440.0           NaN        NaT  51.064732   
           0.0388    239.0    440.0           NaN        NaT  51.064732   
           0.0255    239.0    440.0           NaN        NaT  51.064732   
           0.0100    239.0    440.0           NaN        NaT  51.064732   

                                            longitude  qa_value  \
time       pressure  scanline ground_pixel                        
2021-06-21 1012.0494 182.0    387.0         12.083351       NaN   
           1009.5363 182.0    387.0         12.083351       NaN   
           1006.7900 182.0    387.0         12.083351       NaN   
           1003.7906 182.0    387.0         12.083351       NaN   
           1000.5165 182.0    387.0         12.083351       NaN   
...                                               ...       ...   
           0.0829    239.0    440.0         18.882040       NaN   
           0.0575    239.0    440.0         18.882040       NaN   
           0.0388    239.0    440.0         18.882040       NaN   
           0.0255    239.0    440.0         18.882040       NaN   
           0.0100    239.0    440.0         18.882040       NaN   

                                            TROPOMI_column  kernel_column  \
time       pressure  scanline ground_pixel                                  
2021-06-21 1012.0494 182.0    387.0                    NaN            NaN   
           1009.5363 182.0    387.0                    NaN            NaN   
           1006.7900 182.0    387.0                    NaN            NaN   
           1003.7906 182.0    387.0                    NaN            NaN   
           1000.5165 182.0    387.0                    NaN            NaN   
...                                                    ...            ...   
           0.0829    239.0    440.0                    NaN            NaN   
           0.0575    239.0    440.0                    NaN            NaN   
           0.0388    239.0    440.0                    NaN            NaN   
           0.0255    239.0    440.0                    NaN            NaN   
           0.0100    239.0    440.0                    NaN            NaN   

                                            hybrid  CAMS_component  
time       pressure  scanline ground_pixel                          
2021-06-21 1012.0494 182.0    387.0          137.0    2.984847e+15  
           1009.5363 182.0    387.0          136.0    2.948634e+15  
           1006.7900 182.0    387.0          135.0    2.911252e+15  
           1003.7906 182.0    387.0          134.0    2.872473e+15  
           1000.5165 182.0    387.0          133.0    2.832495e+15  
...                                            ...             ...  
           0.0829    239.0    440.0            5.0    2.370742e+12  
           0.0575    239.0    440.0            4.0    1.760912e+12  
           0.0388    239.0    440.0            3.0    1.269648e+12  
           0.0255    239.0    440.0            2.0    8.514942e+11  
           0.0100    239.0    440.0            1.0    0.000000e+00  

[535572 rows x 9 columns]

#### Interpolate CAMS data to TROPOMI pressures

In [34]:
match_df['CAMS_component'] = match_df['CAMS_component'].interpolate()
match_df

layer delta_time   latitude  \
time       pressure  scanline ground_pixel                                
2021-06-21 1012.0494 182.0    387.0           NaN        NaT  48.498062   
           1009.5363 182.0    387.0           NaN        NaT  48.498062   
           1006.7900 182.0    387.0           NaN        NaT  48.498062   
           1003.7906 182.0    387.0           NaN        NaT  48.498062   
           1000.5165 182.0    387.0           NaN        NaT  48.498062   
...                                           ...        ...        ...   
           0.0829    239.0    440.0           NaN        NaT  51.064732   
           0.0575    239.0    440.0           NaN        NaT  51.064732   
           0.0388    239.0    440.0           NaN        NaT  51.064732   
           0.0255    239.0    440.0           NaN        NaT  51.064732   
           0.0100    239.0    440.0           NaN        NaT  51.064732   

                                            longitude  qa_value  \
time       pressure  scanline ground_pixel                        
2021-06-21 1012.0494 182.0    387.0         12.083351       NaN   
           1009.5363 182.0    387.0         12.083351       NaN   
           1006.7900 182.0    387.0         12.083351       NaN   
           1003.7906 182.0    387.0         12.083351       NaN   
           1000.5165 182.0    387.0         12.083351       NaN   
...                                               ...       ...   
           0.0829    239.0    440.0         18.882040       NaN   
           0.0575    239.0    440.0         18.882040       NaN   
           0.0388    239.0    440.0         18.882040       NaN   
           0.0255    239.0    440.0         18.882040       NaN   
           0.0100    239.0    440.0         18.882040       NaN   

                                            TROPOMI_column  kernel_column  \
time       pressure  scanline ground_pixel                                  
2021-06-21 1012.0494 182.0    387.0                    NaN            NaN   
           1009.5363 182.0    387.0                    NaN            NaN   
           1006.7900 182.0    387.0                    NaN            NaN   
           1003.7906 182.0    387.0                    NaN            NaN   
           1000.5165 182.0    387.0                    NaN            NaN   
...                                                    ...            ...   
           0.0829    239.0    440.0                    NaN            NaN   
           0.0575    239.0    440.0                    NaN            NaN   
           0.0388    239.0    440.0                    NaN            NaN   
           0.0255    239.0    440.0                    NaN            NaN   
           0.0100    239.0    440.0                    NaN            NaN   

                                            hybrid  CAMS_component  
time       pressure  scanline ground_pixel                          
2021-06-21 1012.0494 182.0    387.0          137.0    2.984847e+15  
           1009.5363 182.0    387.0          136.0    2.948634e+15  
           1006.7900 182.0    387.0          135.0    2.911252e+15  
           1003.7906 182.0    387.0          134.0    2.872473e+15  
           1000.5165 182.0    387.0          133.0    2.832495e+15  
...                                            ...             ...  
           0.0829    239.0    440.0            5.0    2.370742e+12  
           0.0575    239.0    440.0            4.0    1.760912e+12  
           0.0388    239.0    440.0            3.0    1.269648e+12  
           0.0255    239.0    440.0            2.0    8.514942e+11  
           0.0100    239.0    440.0            1.0    0.000000e+00  

[535572 rows x 9 columns]

#### Show vertical interpolation for one location

In [35]:
# Query to get data for one location
small = match_df.query('scanline == 182 and ground_pixel == 387')

# Get pressure data
all_pressures = small.index.get_level_values(1).to_numpy()
CAMS_pressures = CAMS_levels['pf [hPa]'].to_numpy()

# Show in black original values and in red the interpolated pressures
diff_colors = np.where(np.isin(all_pressures, CAMS_pressures), 'black', 
                       np.where(~np.isin(all_pressures, CAMS_pressures), 'red', 'yellow'))

# Show component vs. pressures
plt.scatter(small['CAMS_component'], all_pressures, c = diff_colors, s = 10)

# Revert yaxis to have surface pressure on the bottom
ax = plt.gca()
ax.set_ylim(ax.get_ylim()[::-1])
ax.set_xlabel(f'{component_nom} (kg/m²)', fontsize = 18)
ax.set_ylabel('Pressure (hPa)', fontsize = 18)
plt.show()

#### Drop CAMS pressure levels rows
They were meant to be used only for the interpolation

In [36]:
match_df = match_df.drop(CAMS_levels['pf [hPa]'].to_numpy(), level = 'pressure')
match_df

layer              delta_time  \
time       pressure   scanline ground_pixel                                  
2021-06-21 954.330811 182.0    387.0           0.0 2021-06-21 12:42:53.473   
           941.848755 182.0    387.0           1.0 2021-06-21 12:42:53.473   
           919.881226 182.0    387.0           2.0 2021-06-21 12:42:53.473   
           887.258057 182.0    387.0           3.0 2021-06-21 12:42:53.473   
           840.520691 182.0    387.0           4.0 2021-06-21 12:42:53.473   
...                                            ...                     ...   
           6.257802   239.0    440.0          29.0 2021-06-21 12:43:41.351   
           3.094656   239.0    440.0          30.0 2021-06-21 12:43:41.351   
           1.304296   239.0    440.0          31.0 2021-06-21 12:43:41.351   
           0.440648   239.0    440.0          32.0 2021-06-21 12:43:41.351   
           0.093045   239.0    440.0          33.0 2021-06-21 12:43:41.351   

                                              latitude  longitude  qa_value  \
time       pressure   scanline ground_pixel                                   
2021-06-21 954.330811 182.0    387.0         48.498062  12.083351       1.0   
           941.848755 182.0    387.0         48.498062  12.083351       1.0   
           919.881226 182.0    387.0         48.498062  12.083351       1.0   
           887.258057 182.0    387.0         48.498062  12.083351       1.0   
           840.520691 182.0    387.0         48.498062  12.083351       1.0   
...                                                ...        ...       ...   
           6.257802   239.0    440.0         51.064732  18.882040       1.0   
           3.094656   239.0    440.0         51.064732  18.882040       1.0   
           1.304296   239.0    440.0         51.064732  18.882040       1.0   
           0.440648   239.0    440.0         51.064732  18.882040       1.0   
           0.093045   239.0    440.0         51.064732  18.882040       1.0   

                                             TROPOMI_column  kernel_column  \
time       pressure   scanline ground_pixel                                  
2021-06-21 954.330811 182.0    387.0           2.028193e+15       0.247081   
           941.848755 182.0    387.0           2.028193e+15       0.266311   
           919.881226 182.0    387.0           2.028193e+15       0.448114   
           887.258057 182.0    387.0           2.028193e+15       0.724136   
           840.520691 182.0    387.0           2.028193e+15       0.844962   
...                                                     ...            ...   
           6.257802   239.0    440.0           2.239043e+15       0.000000   
           3.094656   239.0    440.0           2.239043e+15       0.000000   
           1.304296   239.0    440.0           2.239043e+15       0.000000   
           0.440648   239.0    440.0           2.239043e+15       0.000000   
           0.093045   239.0    440.0           2.239043e+15       0.000000   

                                             hybrid  CAMS_component  
time       pressure   scanline ground_pixel                          
2021-06-21 954.330811 182.0    387.0            NaN    2.326352e+15  
           941.848755 182.0    387.0            NaN    2.257543e+15  
           919.881226 182.0    387.0            NaN    2.035385e+15  
           887.258057 182.0    387.0            NaN    1.802982e+15  
           840.520691 182.0    387.0            NaN    1.682416e+15  
...                                             ...             ...  
           6.257802   239.0    440.0            NaN    2.956322e+14  
           3.094656   239.0    440.0            NaN    9.133268e+13  
           1.304296   239.0    440.0            NaN    2.873254e+13  
           0.440648   239.0    440.0            NaN    1.401522e+13  
           0.093045   239.0    440.0            NaN    2.793307e+12  

[106488 rows x 9 columns]

#### Show only interpolated values for one location

In [37]:
# Query to get data for one location
small = match_df.query('scanline == 182 and ground_pixel == 387')

# Get pressure data
TROPOMI_pressures = small.index.get_level_values(1).to_numpy()

# Show component vs. pressures
plt.scatter(small['CAMS_component'], TROPOMI_pressures, c = 'red', s = 10)

# Revert yaxis to have surface pressure on the bottom
ax = plt.gca()
ax.set_ylim(ax.get_ylim()[::-1])
ax.set_xlabel(f'{component_nom} (kg/m²)', fontsize = 18)
ax.set_ylabel('Pressure (hPa)', fontsize = 18)
plt.show()

#### Calculate CAMS columns

In [38]:
# Calculate values to generate CAMS column to sum in the next step
match_df['CAMS_column'] = match_df.apply(lambda row: row['CAMS_component'] * 
                                                     row['kernel_column'], 
                                                     axis = 1)

In [39]:
match_df = match_df.set_index('layer', append = True)
match_df = match_df.reset_index('pressure')
match_df

pressure              delta_time  \
time       scanline ground_pixel layer                                       
2021-06-21 182.0    387.0        0.0    954.330811 2021-06-21 12:42:53.473   
                                 1.0    941.848755 2021-06-21 12:42:53.473   
                                 2.0    919.881226 2021-06-21 12:42:53.473   
                                 3.0    887.258057 2021-06-21 12:42:53.473   
                                 4.0    840.520691 2021-06-21 12:42:53.473   
...                                            ...                     ...   
           239.0    440.0        29.0     6.257802 2021-06-21 12:43:41.351   
                                 30.0     3.094656 2021-06-21 12:43:41.351   
                                 31.0     1.304296 2021-06-21 12:43:41.351   
                                 32.0     0.440648 2021-06-21 12:43:41.351   
                                 33.0     0.093045 2021-06-21 12:43:41.351   

                                         latitude  longitude  qa_value  \
time       scanline ground_pixel layer                                   
2021-06-21 182.0    387.0        0.0    48.498062  12.083351       1.0   
                                 1.0    48.498062  12.083351       1.0   
                                 2.0    48.498062  12.083351       1.0   
                                 3.0    48.498062  12.083351       1.0   
                                 4.0    48.498062  12.083351       1.0   
...                                           ...        ...       ...   
           239.0    440.0        29.0   51.064732  18.882040       1.0   
                                 30.0   51.064732  18.882040       1.0   
                                 31.0   51.064732  18.882040       1.0   
                                 32.0   51.064732  18.882040       1.0   
                                 33.0   51.064732  18.882040       1.0   

                                        TROPOMI_column  kernel_column  hybrid  \
time       scanline ground_pixel layer                                          
2021-06-21 182.0    387.0        0.0      2.028193e+15       0.247081     NaN   
                                 1.0      2.028193e+15       0.266311     NaN   
                                 2.0      2.028193e+15       0.448114     NaN   
                                 3.0      2.028193e+15       0.724136     NaN   
                                 4.0      2.028193e+15       0.844962     NaN   
...                                                ...            ...     ...   
           239.0    440.0        29.0     2.239043e+15       0.000000     NaN   
                                 30.0     2.239043e+15       0.000000     NaN   
                                 31.0     2.239043e+15       0.000000     NaN   
                                 32.0     2.239043e+15       0.000000     NaN   
                                 33.0     2.239043e+15       0.000000     NaN   

                                        CAMS_component   CAMS_column  
time       scanline ground_pixel layer                                
2021-06-21 182.0    387.0        0.0      2.326352e+15  5.747976e+14  
                                 1.0      2.257543e+15  6.012088e+14  
                                 2.0      2.035385e+15  9.120839e+14  
                                 3.0      1.802982e+15  1.305604e+15  
                                 4.0      1.682416e+15  1.421578e+15  
...                                                ...           ...  
           239.0    440.0        29.0     2.956322e+14  0.000000e+00  
                                 30.0     9.133268e+13  0.000000e+00  
                                 31.0     2.873254e+13  0.000000e+00  
                                 32.0     1.401522e+13  0.000000e+00  
                                 33.0     2.793307e+12  0.000000e+00  

[106488 rows x 10 columns]

In [40]:
match_ds = match_df.to_xarray()
match_ds

<xarray.Dataset>
Dimensions:         (ground_pixel: 54, layer: 34, scanline: 58, time: 1)
Coordinates:
  * time            (time) datetime64[ns] 2021-06-21
  * scanline        (scanline) float64 182.0 183.0 184.0 ... 237.0 238.0 239.0
  * ground_pixel    (ground_pixel) float64 387.0 388.0 389.0 ... 439.0 440.0
  * layer           (layer) float64 0.0 1.0 2.0 3.0 4.0 ... 30.0 31.0 32.0 33.0
Data variables:
    pressure        (time, scanline, ground_pixel, layer) float64 954.3 ... 0...
    delta_time      (time, scanline, ground_pixel, layer) datetime64[ns] 2021...
    latitude        (time, scanline, ground_pixel, layer) float64 48.5 ... 51.06
    longitude       (time, scanline, ground_pixel, layer) float64 12.08 ... 1...
    qa_value        (time, scanline, ground_pixel, layer) float32 1.0 ... 1.0
    TROPOMI_column  (time, scanline, ground_pixel, layer) float64 2.028e+15 ....
    kernel_column   (time, scanline, ground_pixel, layer) float32 0.2471 ... 0.0
    hybrid          (time, scanline, ground_pixel, layer) float64 nan ... nan
    CAMS_component  (time, scanline, ground_pixel, layer) float64 2.326e+15 ....
    CAMS_column     (time, scanline, ground_pixel, layer) float64 5.748e+14 ....

In [41]:
# Read latitudes and longitudes from data array
latitude = match_ds.latitude.mean(dim = 'layer')
longitude = match_ds.longitude.mean(dim = 'layer')

In [42]:
# Get sum of CAMS data of each layer to get column data
CAMS = match_ds.CAMS_column.sum(dim = 'layer', skipna = False)
CAMS = CAMS.assign_coords(latitude = latitude, longitude = longitude)
CAMS

<xarray.DataArray 'CAMS_column' (time: 1, scanline: 58, ground_pixel: 54)>
array([[[3.31029610e+16, 3.28891678e+16, 3.31576196e+16, ...,
         2.99164330e+16, 3.05179638e+16, 3.07633910e+16],
        [3.35748362e+16, 3.31846368e+16, 3.32610523e+16, ...,
         2.96969967e+16, 3.00312436e+16, 3.08236294e+16],
        [3.31896712e+16, 3.31420705e+16, 3.32773584e+16, ...,
         2.94367974e+16, 2.94072921e+16, 2.97893573e+16],
        ...,
        [3.35510085e+16, 3.38154271e+16, 3.44947408e+16, ...,
         3.55027743e+16, 3.58483924e+16, 3.64457902e+16],
        [3.35611458e+16, 3.34426897e+16, 3.40590607e+16, ...,
         3.66153478e+16, 3.64480519e+16, 3.69225580e+16],
        [3.40904757e+16, 3.45197711e+16, 3.41369545e+16, ...,
         3.69480015e+16, 3.71607274e+16, 3.68672342e+16]]])
Coordinates:
  * time          (time) datetime64[ns] 2021-06-21
  * scanline      (scanline) float64 182.0 183.0 184.0 ... 237.0 238.0 239.0
  * ground_pixel  (ground_pixel) float64 387.0 388.0 389.0 ... 438.0 439.0 440.0
    latitude      (time, scanline, ground_pixel) float64 48.5 48.5 ... 51.06
    longitude     (time, scanline, ground_pixel) float64 12.08 12.19 ... 18.88

In [43]:
# Get mean of TROPOMI data of each layer (it must be equal)
TROPOMI = match_ds.TROPOMI_column.mean(dim = 'layer', skipna = False)
TROPOMI = TROPOMI.assign_coords(latitude = latitude, longitude = longitude)
TROPOMI

<xarray.DataArray 'TROPOMI_column' (time: 1, scanline: 58, ground_pixel: 54)>
array([[[2.02819298e+15, 1.71451335e+15, 1.93141430e+15, ...,
         1.45532656e+15, 1.10716654e+15, 9.53418712e+14],
        [2.22845279e+15, 2.73555022e+15, 2.09747668e+15, ...,
         1.99942064e+15, 1.86558042e+15, 1.11615882e+15],
        [1.05144602e+15, 1.96068140e+15, 1.97075773e+15, ...,
         2.14389423e+15, 2.01104864e+15, 1.98499780e+15],
        ...,
        [5.07239347e+14, 1.09923472e+15, 6.99258017e+14, ...,
         1.49555126e+15, 1.66586498e+15, 2.24578762e+15],
        [1.08164495e+15, 1.22188343e+15, 5.40617766e+14, ...,
         2.01081129e+15, 2.66609743e+15, 1.60509531e+15],
        [1.05369640e+15, 1.23793594e+15, 1.62317257e+15, ...,
         1.89834376e+15, 2.21357157e+15, 2.23904295e+15]]])
Coordinates:
  * time          (time) datetime64[ns] 2021-06-21
  * scanline      (scanline) float64 182.0 183.0 184.0 ... 237.0 238.0 239.0
  * ground_pixel  (ground_pixel) float64 387.0 388.0 389.0 ... 438.0 439.0 440.0
    latitude      (time, scanline, ground_pixel) float64 48.5 48.5 ... 51.06
    longitude     (time, scanline, ground_pixel) float64 12.08 12.19 ... 18.88

## 6. Merge data

In [44]:
merged_ds = xr.merge([CAMS, TROPOMI])
merged_ds

<xarray.Dataset>
Dimensions:         (ground_pixel: 54, scanline: 58, time: 1)
Coordinates:
  * time            (time) datetime64[ns] 2021-06-21
  * scanline        (scanline) float64 182.0 183.0 184.0 ... 237.0 238.0 239.0
  * ground_pixel    (ground_pixel) float64 387.0 388.0 389.0 ... 439.0 440.0
    latitude        (time, scanline, ground_pixel) float64 48.5 48.5 ... 51.06
    longitude       (time, scanline, ground_pixel) float64 12.08 12.19 ... 18.88
Data variables:
    CAMS_column     (time, scanline, ground_pixel) float64 3.31e+16 ... 3.687...
    TROPOMI_column  (time, scanline, ground_pixel) float64 2.028e+15 ... 2.23...

In [45]:
merged_df = merged_ds.to_dataframe()
merged_df

latitude  longitude   CAMS_column  \
ground_pixel scanline time                                             
387.0        182.0    2021-06-21  48.498062  12.083351  3.310296e+16   
             183.0    2021-06-21  48.546898  12.072847  3.357484e+16   
             184.0    2021-06-21  48.595726  12.062323  3.318967e+16   
             185.0    2021-06-21  48.644573  12.051785  3.363705e+16   
             186.0    2021-06-21  48.693401  12.041157  3.338792e+16   
...                                     ...        ...           ...   
440.0        235.0    2021-06-21  50.872116  18.894257  3.651490e+16   
             236.0    2021-06-21  50.920200  18.891136  3.605956e+16   
             237.0    2021-06-21  50.968403  18.888002  3.644579e+16   
             238.0    2021-06-21  51.016579  18.885010  3.692256e+16   
             239.0    2021-06-21  51.064732  18.882040  3.686723e+16   

                                  TROPOMI_column  
ground_pixel scanline time                        
387.0        182.0    2021-06-21    2.028193e+15  
             183.0    2021-06-21    2.228453e+15  
             184.0    2021-06-21    1.051446e+15  
             185.0    2021-06-21    1.280466e+15  
             186.0    2021-06-21    2.157828e+15  
...                                          ...  
440.0        235.0    2021-06-21    1.962260e+15  
             236.0    2021-06-21    2.333197e+15  
             237.0    2021-06-21    2.245788e+15  
             238.0    2021-06-21    1.605095e+15  
             239.0    2021-06-21    2.239043e+15  

[3132 rows x 4 columns]

## 7. Visualize data

In [46]:
units = component_nom + ' (molecules/cm2)'
projection = ccrs.PlateCarree()

for t in range(TROPOMI.time.size):
    
    fig, axs = plt.subplots(1, 2, figsize = (20, 5), subplot_kw = {'projection': projection})
    
    """
    # Define vmin
    if (np.nanmin(TROPOMI.isel(time = t)) < np.nanmin(CAMS.isel(time = t))) and np.nanmin(TROPOMI.isel(time = t)) >= 0:
        vmin = np.nanmin(TROPOMI.isel(time = t))
    else:
        vmin = np.nanmin(CAMS.isel(time = t))
    
    # Define vmax
    if np.nanmax(TROPOMI.isel(time = t)) < np.nanmax(CAMS.isel(time = t)):
        vmax = np.nanmax(CAMS.isel(time = t))
    else:
        vmax = np.nanmax(TROPOMI.isel(time = t))
    """

    # First plot - CAMS
    array = CAMS.isel(time = t)
    long_name = 'CAMS (Forecast)'
    visualize_pcolormesh(
                        fig = fig, axs = axs[0],
                        data_array = array.fillna(-999),
                        longitude = array.longitude,
                        latitude = array.latitude,
                        projection = ccrs.PlateCarree(),
                        color_scale = 'coolwarm',
                        long_name = long_name,
                        units = units,
                        vmin = 0, 
                        vmax = np.nanmax(CAMS.isel(time = t)), 
                        set_global = False,
                        lonmin = bbox[0][0],
                        lonmax = bbox[1][0],
                        latmin = bbox[0][1],
                        latmax = bbox[1][1]
                        )

    # Second plot - TROPOMI
    array = TROPOMI.isel(time = t)
    long_name = 'TROPOMI (NRT)'
    visualize_pcolormesh(
                        fig = fig, axs = axs[1],
                        data_array = array.fillna(-999),
                        longitude = array.longitude,
                        latitude = array.latitude,
                        projection = ccrs.PlateCarree(),
                        color_scale = 'coolwarm',
                        long_name = long_name,
                        units = units,
                        vmin = 0,  
                        vmax = np.nanmax(TROPOMI.isel(time = t)), 
                        set_global = False,
                        lonmin = bbox[0][0],
                        lonmax = bbox[1][0],
                        latmin = bbox[0][1],
                        latmax = bbox[1][1]
                        )

    fig.suptitle(f'DISTRIBUTION OF {component_nom} (Estimated time: 12:00:00 on 21-06-2021)',
                  fontsize = 18)
    plt.show()

/home/alba/adc-toolbox/env-new/lib/python3.8/site-packages/cartopy/mpl/geoaxes.py:1702: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  X, Y, C, shading = self._pcolorargs('pcolormesh', *args,



## References
### Data sources
* CAMS global atmospheric composition forecasts (https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-global-atmospheric-composition-forecasts?tab=overview)
* TROPOMI (https://s5phub.copernicus.eu/dhus)

### Background
* TROPOMI user manual (https://sentinels.copernicus.eu/documents/247904/3119978/Sentinel-5P-Level-2-Input-Output-Data-Definition)

### External functions
* Interpolation (http://christopherbull.com.au/python/scipy-interpolate-griddata/)
* Visualization function (https://gitlab.eumetsat.int/eumetlab/atmosphere/atmosphere/-/blob/master/functions.ipynb)
* Mask NaN values (https://matplotlib.org/2.0.2/examples/pylab_examples/image_masked.html)
* Convert units (https://confluence.ecmwf.int/pages/viewpage.action?pageId=153391710)
* Install eccodes (https://gist.github.com/MHBalsmeier/a01ad4e07ecf467c90fad2ac7719844a)